순서
1. 미리 만들어둔 dataframe가져오기
2. feats지정
3. lgbm과 같은 boosting 실행
4. feature importance 확인
5. shap확인
6. valid 및 저장

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import warnings
warnings.filterwarnings('ignore')
import random
seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from lightgbm import LGBMClassifier
import lightgbm as lgb


In [2]:
df = pd.read_csv('/data/ephemeral/data/preprocedding.csv', index_col=0)

In [3]:
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)
def custom_train_test_split(df, ratio=0.7, split=True):

    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)

    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [ ]:
df.columns

In [ ]:
FEATS1 = [
       'KnowledgeTag', 'elapsed', 'cumulative', 'paper_number',
       'mean_elaseped_tag', 'meanelapsed_tag_cor', 'meanelapsed_tag_wrong',
       'mean_elapsed_paper_num', 'mean_elapsed_paper_num_cor',
       'mean_elapsed_paper_num_wrong', 'paper_type',
       'paper_subtype', 'paper_number_percent', 'paper_type_percent',
       'KnowledgeTag_percent', 'paper_number_count', 'tag_count', 'paper_pos']

In [ ]:
FEATS2 = ['KnowledgeTag', 'dataset', 'elapsed', 'cumulative', 'paper_number',
       'mean_elaseped_tag', 'meanelapsed_tag_cor', 'meanelapsed_tag_wrong',
       'mean_elapsed_paper_num', 'mean_elapsed_paper_num_cor',
       'mean_elapsed_paper_num_wrong', 'paper_type',
       'paper_subtype', 'paper_number_percent', 'paper_type_percent',
       'KnowledgeTag_percent', 'paper_number_count', 'tag_count', 'paper_pos',
    'user_answer_mean',
       'reverse_answer_cumsum', 'testid_cumsum', 'tag_cumsum',
       'relative_answer_score', 'elapsed_cumsum', 'normalized_elapsed',
       'elapsed_type_mean', 'rolling_mean_time']

In [4]:
FEATS3 = ['KnowledgeTag', 'dataset', 'elapsed', 'cumulative', 'paper_number',
       'mean_elaseped_tag', 'meanelapsed_tag_cor', 'meanelapsed_tag_wrong',
       'mean_elapsed_paper_num', 'mean_elapsed_paper_num_cor',
       'mean_elapsed_paper_num_wrong', 'paper_type',
       'paper_subtype', 'paper_number_percent', 'paper_type_percent',
       'KnowledgeTag_percent', 'paper_number_count', 'tag_count', 'paper_pos',
        'user_answer_mean', 'user_ass_cumcount',
       'reverse_answer_cumsum', 'testid_cumsum', 'tag_cumsum',
       'relative_answer_score', 'elapsed_cumsum', 'normalized_elapsed',
       'elapsed_type_mean', 'rolling_mean_time', 'problem_correct_per_user',
       'problem_solved_per_user',
       'acc_tag_count_per_user', 'correct_answer_per_as',
       'correct_answer_per_te', 'acc_count_per_as', 'acc_count_per_te',
       'acc_answerRate_per_as', 'acc_answerRate_per_te', 'acc_elapsed_per_as',
       'acc_elapsed_per_te']

In [5]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(df[df['answerCode'] != -1], test_size = 0.2, random_state = seed, shuffle = True)

y_train = train["answerCode"]
train = train.drop(["answerCode"], axis=1)

y_valid = valid["answerCode"]
valid = valid.drop(["answerCode"], axis=1)

print(train.shape, y_train.shape, valid.shape, y_valid.shape)

(2020764, 47) (2020764,) (505192, 47) (505192,)


# optuna

wandb로 실시간 결과 확인 코드 추가

In [7]:
!pip install optuna
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.6 MB/s eta 0:00:00


In [12]:
from lightgbm import early_stopping
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgb

import wandb
from wandb.lightgbm import wandb_callback

wandb.login()



sampler = TPESampler(seed=10)
def objective(trial):
    wandb.init(project="optuna_lightgbm", reinit=True)
    
    param = {
        'objective': 'binary', # 회귀
        'verbose': -1,
        'metric': 'auc', 
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
    gbm = lgb.train(
    param,
    lgb.Dataset(train[FEATS3], label=y_train),
    valid_sets = [lgb.Dataset(valid[FEATS3], label=y_valid)],
    num_boost_round = 100,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
                                  wandb_callback()]
    )
    
    preds = gbm.predict(valid[FEATS3])
    roc_auc = roc_auc_score(y_valid, preds)
    
    wandb.log({"roc_auc": roc_auc})
    
    return roc_auc


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
[I 2024-01-23 12:09:35,546] A new study created in memory with name: no-name-034aba60-ca35-43bf-a5b7-e91b408087f7


Training until validation scores don't improve for 50 rounds


[W 2024-01-23 12:09:54,736] Trial 0 failed with parameters: {'num_leaves': 409, 'colsample_bytree': 0.9221964459013311, 'reg_alpha': 0.7608049464159028, 'reg_lambda': 5.362886387231978, 'max_depth': 13, 'learning_rate': 1.0629481745297198e-07, 'n_estimators': 364, 'min_child_samples': 9, 'subsample': 0.9418988450454984} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/data/ephemeral/miniconda3/envs/dkt/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_656273/3377442996.py", line 31, in objective
    gbm = lgb.train(
  File "/data/ephemeral/miniconda3/envs/dkt/lib/python3.10/site-packages/lightgbm/engine.py", line 276, in train
    booster.update(fobj=fobj)
  File "/data/ephemeral/miniconda3/envs/dkt/lib/python3.10/site-packages/lightgbm/basic.py", line 3891, in update
    _safe_call(_LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterrupt
[W 2024-01-23 12:09:

KeyboardInterrupt: 

최적화 하이퍼파라미터로 훈련

In [ ]:
best_params = study.best_trial.params
best_params['objective'] = 'binary'
best_params['metric'] = 'auc'

model = lgb.train(
    best_params,
    lgb.Dataset(train[FEATS3], label=y_train),
    valid_sets=[lgb.Dataset(valid[FEATS3], label=y_valid)],
    num_boost_round=1000,
    early_stopping_rounds=50
)

In [ ]:
#  LGBoost 모델 추론
preds = model.predict_proba(valid[FEATS3])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f"VALID AUC : {auc} ACC : {acc}\n")

feature importance 확인

In [ ]:
feature_importances = model.feature_importances_

# Feature Importance 시각화
plt.figure(figsize=(10, 6))
lgb.plot_importance(model, max_num_features=30)  # 상위 30개 feature 시각화
plt.title("Feature Importances")
plt.show()


In [ ]:
%pip install shap

In [ ]:
import shap

# SHAP 값을 계산
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(valid[FEATS3])

# SHAP 요약 플롯 시각화
shap.summary_plot(shap_values, valid[FEATS3], plot_type="bar")

최종 output 확인

In [ ]:
test_df = df[df.dataset == 2]
# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df["userID"] != test_df["userID"].shift(-1)]
# DROP ANSWERCODE
test_df = test_df.drop(["answerCode"], axis=1)
# MAKE PREDICTION
total_probs = model.predict_proba(test_df[FEATS])[:,-1]

In [ ]:
# SAVE OUTPUT
output_dir = '/data/ephemeral/code/lightgbm/result/'
write_path = os.path.join(output_dir, "submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(preds):
        w.write('{},{}\n'.format(id,p))

일반적인 lgbm

In [ ]:
model = LGBMClassifier(n_estimators=1000,
    #                    objective = 'binary',
    # hum_boost_round = 500,
    # early_stopping_rounds = 100,
    # boost_from_average=False
)


model.fit(X=train[FEATS3],
          y=y_train,
          eval_set=[(valid[FEATS3], y_valid)],
          eval_metric="auc",
          # early_stopping_rounds=100,
         )

In [ ]:
feature_importances = model.feature_importances_

# Feature Importance 시각화
plt.figure(figsize=(10, 6))
lgb.plot_importance(model, max_num_features=30)  # 상위 30개 feature 시각화
plt.title("Feature Importances")
plt.show()


In [ ]:
import shap

# SHAP 값을 계산
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(valid[FEATS3])

# SHAP 요약 플롯 시각화
shap.summary_plot(shap_values, valid[FEATS3], plot_type="bar")

In [ ]:
test_df = df[df.dataset == 2]
# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df["userID"] != test_df["userID"].shift(-1)]
# DROP ANSWERCODE
test_df = test_df.drop(["answerCode"], axis=1)
# MAKE PREDICTION
total_probs = model.predict_proba(test_df[FEATS])[:,-1]

In [ ]:
model = LGBMClassifier(n_estimators=1000,
    #                    objective = 'binary',
    # hum_boost_round = 500,
    # early_stopping_rounds = 100,
    # boost_from_average=False
)


model.fit(X=train[FEATS3],
          y=y_train,
          eval_set=[(valid[FEATS3], y_valid)],
          eval_metric="auc",
          # early_stopping_rounds=100,
         )